In [1]:
import csv
import json
import os
from collections import defaultdict

In [2]:
folder = 'results/'
remove_slot_value = False
if remove_slot_value:
    baselines = ['FVN-', 'FVN without Control', 'FVN']
    files = ['simple_VQVAE_personage_LSTM_delex_CB512EL1TMT62', 'focused_VQVAE_personage_LSTM_delex_CB1024EL3TMT62', 'controlled_VQVAE_personage_LSTM_delex_CB512EL1TMT62']
else:
    baselines = ['FVN without Control', 'FVN']
    files = ['focused_VQVAE_personage_LSTM_CB1024EL3TMT62', 'controlled_VQVAE_personage_LSTM_CB512EL1TMT62']

In [3]:
def read_result_data(fn):
    file = open(fn, 'r')
    meta = []
    while True:
        line = file.readline()
        if 'START_CSV_SECTION' in line:
            break
    meta.append(line)
    reader = csv.DictReader(file)
    data = [_ for _ in reader]
    file.close()
    return data

def pack_dial(data):
    dials = {}
    for turn in data:
        dial_id = int(turn['id'])
        if dial_id not in dials:
            dials[dial_id] = []
        dials[dial_id].append(turn)
    return dials

def dump(dats):
        for k, v in data.items():
            dump_file.write('{}\t{}\n'.format(k, v))


In [7]:
def cleantext(t):
    s_tokens = ['<go> ', ' EOS']
    for s in s_tokens:
        t = t.replace(s, '')
    return t

output = defaultdict(dict)
for baseline, file in zip(baselines, files):
    data = read_result_data(folder+file+'.csv')
    print (baseline, file, len(data))
    for d in data:
        _id = int(d['id'])
        if  _id not in output:
            output[_id]['id']=d['id']
            output[_id]['slot_value']=d['slot_value']
            output[_id]['personality']=d['personality']
        if remove_slot_value:
            output[_id]['ground_truth']=cleantext(d['delex_text'])
            output[_id][baseline]=cleantext(d['pred_delex_text'])
        else:
            output[_id]['ground_truth']=cleantext(d['text'])
            output[_id][baseline]=cleantext(d['pred_text'])
        
     

FVN without Control focused_VQVAE_personage_LSTM_CB1024EL3TMT62 1390
FVN controlled_VQVAE_personage_LSTM_CB512EL1TMT62 1390
let 's see , nameVariable ... oh , it is a indian restaurant and city centre and moderately priced near nearVariable with a mediocre rating , also it is a somewhat family friendly , and it 's a pub , alright ?
oh god i do n't know . nameVariable is a chinese place , damn family friendly and cheap in riverside and it has like , an average rating , also it is a of near nearVariable , also it is a pub .
nameVariable is a indian place , it is damn expensive and it 's near nearVariable . it is a restaurant . it is in riverside . it is kid friendly .
nameVariable is an english place , also it is near nearVariable . it is in city centre . it is a restaurant . it is n't damn family friendly .
let 's see , nameVariable ... well , i see it is a pub with a price range of £20-25 , also it is an italian restaurant and in riverside family friendly near nearVariable .
let 's see

In [8]:
if remove_slot_value:
    result_file = open('delex_result4humanevaluation.csv', 'w')
else:
    result_file = open('nondelex_result4humanevaluation.csv', 'w')
writer = csv.DictWriter(result_file, fieldnames=['id', 'slot_value', 'personality', 'ground_truth'] + baselines)
writer.writeheader()
for i, result in sorted(output.items()):
    writer.writerow(result)
result_file.close()   